In [3]:
!pip install -q -r requirements.txt

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.4 MB/s eta 0:00:00


In [ ]:
import os
import json
import torch
import pickle
import random
import string
import logging
import shutil
import numpy as np
import torch.nn as nn
from tqdm import tqdm
from CiteHeteroSum.utils import *
from CiteHeteroSum.CiteHeteroSum_model.Loss import Triplet
from CiteHeteroSum.CiteHeteroSum_model.Testing_Process import test_e2e
from CiteHeteroSum.CiteHeteroSum_model.Graph_Encoder import StepWiseGraphConvLayer, Contrast_Encoder, MLP, End2End_Encoder
from config import args

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/228k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/442M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [5]:
# output config
model_save_root_path = f'data/checkpoint/'
best_r2, best_c_loss, best_s_loss = 0, 10000, 10000
history = {'loss': [], 'val_loss': []}

# seed and device
seed_everything(args['seed'])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

## CONFIG

In [6]:
scenario = "1"
valid_scenarios = ["1", "2", "3"]

if scenario not in valid_scenarios:
    raise ValueError(f"Invalid scenario: {scenario}. Must be one of {valid_scenarios}")

In [7]:
import json
with open("./config.json", "r") as f:
    config_data = json.load(f)

test_file = config_data['Test'][f"scenario{scenario}"]["output"]
model_version = "_".join(f"scenario{scenario}")

c_file_path = config_data['checkpoint'][f"scenario{scenario}"]["c"]
e_file_path = c_file_path.replace("/c_", "/e_")

In [8]:
with open(test_file, "rb") as f:
    testGraphs = pickle.load(f)

In [9]:
import logging

logging.basicConfig(filename='app.log', level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Inference and Evaluation

In [10]:
c_graph_encoder = StepWiseGraphConvLayer(in_dim=768, out_dim=args['hidden'], hid_dim=args['hidden'],
                                         dropout_p=args['dropout'], act=nn.LeakyReLU(), nheads=args['heads'], iter=1).to(device)

contrast_filter = Contrast_Encoder(c_graph_encoder, args['hidden'], dropout_p=args['dropout']).to(device)
summarization_encoder = End2End_Encoder(768, args['hidden'], args['dropout']).to(device)

optimizer = torch.optim.Adam([ {'params': summarization_encoder.parameters()},
                            {'params': contrast_filter.parameters()}], lr=args['lr'], weight_decay=1e-5)

loss_method = Triplet(margin=args['triplet_margin'], topk=args['triplet_topk'])

In [11]:
summarization_encoder.load_state_dict(torch.load(e_file_path, map_location=torch.device('cuda')), strict=False)
contrast_filter.load_state_dict(torch.load(c_file_path, map_location=torch.device('cuda')), strict=False)
model = [contrast_filter, summarization_encoder]

In [12]:
def print_evaluation_scores(rouge_means, bert):
    for rouge_type, metrics in rouge_means.items():
        print(f"{rouge_type.upper()} Scores:")
        for metric, value in metrics.items():
            print(f"  {metric}: {value:.4f}")
        print()

    print("Bert Scores:")
    for metric, value in bert.items():
        print(f"  {metric}: {value:.4f}")

In [13]:
import os
import json

output_dir = "data/output"
os.makedirs(output_dir, exist_ok=True)
predicts, goldens = test_e2e(testGraphs, model, args["summary_max_word_num"], 0, args["kappa"])

with open(os.path.join(output_dir, "predicts.json"), "w", encoding="utf-8") as f_pred:
    json.dump(predicts, f_pred, ensure_ascii=False, indent=2)
with open(os.path.join(output_dir, "goldens.json"), "w", encoding="utf-8") as f_gold:
    json.dump(goldens, f_gold, ensure_ascii=False, indent=2)

In [14]:
rouge_scores = get_rouges(goldens, predicts)
bert_score = get_bert_score(goldens, predicts)
print_evaluation_scores(rouge_scores, bert_score)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/2 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 3.59 seconds, 17.29 sentences/sec
ROUGE1 Scores:
  p: 0.5666
  r: 0.6550
  f: 0.5988

ROUGE2 Scores:
  p: 0.3847
  r: 0.4504
  f: 0.4091

ROUGEL Scores:
  p: 0.4250
  r: 0.4959
  f: 0.4513

Bert Scores:
  precision: 0.8886
  recall: 0.8947
  fmeasure: 0.8916
